In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import MeanShift
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 6 штук по всему миру. Задача выбрать 6 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn


In [ ]:
#Считываем файл с данными
df = pd.read_csv('/content/drive/MyDrive/coursera/checkins.dat', sep='|',header=0, skipinitialspace=True)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
df.head()

,id,user_id,venue_id,latitude,longitude,created_at
1,984301,2041916.0,5222.0,NaN,NaN,2012-04-21 17:39:01
2,984222,15824.0,5222.0,38.895112,-77.036366,2012-04-21 17:43:47
3,984315,1764391.0,5222.0,NaN,NaN,2012-04-21 17:37:18
4,984234,44652.0,5222.0,33.800745,-84.410520,2012-04-21 17:43:43
5,984249,2146840.0,5222.0,NaN,NaN,2012-04-21 17:42:58


Удаляем строки, которые не содержат координат

In [ ]:
df = df.dropna()

In [ ]:
#Для уменьшения времени обучения отберем 100000 строк
df = df[0:100000]

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, возьмем алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Значение bandwidth=0.1  в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

In [ ]:
#Создадим модель на координатах и найдем кластеры
ms = MeanShift(bandwidth = 0.1)
ms.fit(df.iloc[:,3:5])

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, max_iter=300,
          min_bin_freq=1, n_jobs=None, seeds=None)

In [ ]:
#Выведем центройды кластеров 
center = ms.cluster_centers_
center

array([[  40.7177164 ,  -73.99183542],
       [  33.44943805, -112.00213969],
       [  33.44638027, -111.90188756],
       ...,
       [ -37.8229826 ,  145.1811902 ],
       [ -41.2924945 ,  174.7732353 ],
       [ -45.0311622 ,  168.6626435 ]])

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения

In [ ]:
label  = ms.labels_ #получаем метки принадлежности к кластерам
val, counts = np.unique(label, return_counts=True) #получаем сколько точек относятся к каждому кластеру
cluster_count = np.vstack((val,counts)) 
cluster_count = cluster_count.T
filt_cluster = cluster_count[cluster_count[:,1] > 15] #отбираем класстеры, к которым относится более 15 точек

In [ ]:
len(filt_cluster)

592

In [ ]:
# Получим координаты отобранных кластеров
filt_centers = pd.DataFrame(center)
for i, _ in enumerate(center):
    if i not in filt_cluster[:,0]:
        filt_centers.drop(i, inplace=True)

In [ ]:
filt_centers

,0,1
0,40.717716,-73.991835
1,33.449438,-112.002140
2,33.446380,-111.901888
3,41.878244,-87.629843
4,37.688682,-122.409330
...,...,...
684,41.577224,-73.415723
727,41.220398,-73.666619
884,41.618532,-88.445568
1343,39.249469,-77.182127


In [ ]:
coord_and_clusters = df.iloc[:,3:5]
coord_and_clusters['labels'] = label
coord_and_clusters.head()

,latitude,longitude,labels
2,38.895112,-77.036366,5
4,33.800745,-84.410520,7
8,45.523452,-122.676207,30
10,40.764462,-111.904565,65
11,33.448377,-112.074037,1


In [ ]:
#Запишем координаты офисов фирмы
office_coord = np.array([[33.751277,-118.188740],
          [25.867736, -80.324116],
          [51.503016, -0.075479],
          [52.378894, 4.885084],
          [39.366487, 117.036146],
          [-33.868457, 151.205134]])

In [ ]:
#Получим кластеры к которым относятся координаты офисов
office_cluster = ms.predict(office_coord)
office_cluster

array([  51,  419,   58,  370, 1980,  420])

In [ ]:
#получим координаты ближайших подходящих мест вблизи офисов для размещения банеров 
best_place = []
for i, cluster in enumerate(office_cluster):
  if cluster in filt_cluster: #если офис располагается в отобраном кластере, то ищем точку внутри этого кластера
    nearest_coord = coord_and_clusters.loc[coord_and_clusters.labels == cluster] #отбираем точки внутри кластера
    distance = np.sum((nearest_coord.values[:,:2] - office_coord[i])**2, axis=1) #считаем расстояние между координатами
    nearest_coord['distance'] = distance
    best_place.append(nearest_coord.sort_values('distance').values[0,:2]) #сортируем в порядке возрастания и берем точку с наименьшим расстоянием
  else: #ищем точку внутри внутри всех подходящих кластеров
    nearest_coord = coord_and_clusters.loc[coord_and_clusters.labels.isin(filt_centers.index.values)] #отбираем точки для всех подходящих кластеров
    distance = np.sum((nearest_coord.values[:,:2] - office_coord[i])**2, axis=1) #считаем расстояние между координатами
    nearest_coord['distance'] = distance
    best_place.append(nearest_coord.sort_values('distance').values[0,:2]) #сортируем в порядке возрастания и берем точку с наименьшим расстоянием


    

In [ ]:
#выводим наилучшие места для размещения банеров
best_place

[array([  33.8044614, -118.1678456]),
 array([ 25.8650961, -80.3244957]),
 array([51.489112 , -0.1106702]),
 array([52.3730556,  4.8922222]),
 array([ 31.230393, 121.473704]),
 array([-33.873651 , 151.2068896])]